In [ ]:
%%capture
!git clone https://github.com/dpcomp-org/dpcomp_core.git

In [ ]:
%%capture
!pip install diffprivlib

In [ ]:
# Remove default module
%%capture
%cd dpcomp_core
!rm -r dpcomp_core

In [ ]:
# Get zipped new module from Jane's server
%%capture
!wget https://dev.universalities.com/dpcomp_core.zip

In [ ]:
# Unzip Jane's version of the module
%%capture
!unzip dpcomp_core.zip

In [ ]:
# Add requirements so pip does the version matching right
!echo diffprivlib >> resources/requirements.txt
!echo pandas >> resources/requirements.txt

In [ ]:
# Install pip requirements
%%capture
!pip install -r resources/requirements.txt

In [ ]:
# Install swig
%%capture
!apt-get install swig

In [ ]:
# Run C++ setup for DAWA
%%capture
%cd dpcomp_core/algorithm/dawa/
!bash setup.sh
%cd ../../../

In [ ]:
import os
os.getcwd()

'/content/dpcomp_core'

In [ ]:
%cd dpcomp_core/algorithm/ahp/
!bash setup.sh
%cd ../../../

/content/dpcomp_core/dpcomp_core/algorithm/ahp
rm: cannot remove 'ahp_fast.c': No such file or directory
rm: cannot remove '*.so': No such file or directory
Compiling lib/ahp_fast.pyx because it changed.
[1/1] Cythonizing lib/ahp_fast.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/dpcomp_core/dpcomp_core/algorithm/ahp/lib/ahp_fast.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'ahp_fast' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/lib
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c lib/ahp_fast.c

In [ ]:
from __future__ import division
from __future__ import print_function
from past.utils import old_div
from dpcomp_core.algorithm import dawa
from dpcomp_core.algorithm import AG
from dpcomp_core.algorithm import ahp
from dpcomp_core import dataset
from dpcomp_core import workload
import numpy as np

In [ ]:
def runDAWA(data, bin, epsilon):
  domain = (bin,bin)
  epsilon = epsilon


  seed = 1
  shape_list = [(5,5),(10,10)]
  size = 5000

  w = workload.RandomRange(shape_list=shape_list, 
                         domain_shape=domain, 
                         size=size, 
                         seed=seed)

  a = dawa.dawa2D_engine()
  H, xedges, yedges = np.histogram2d(data['x'], data['y'], bins = bin)
  H = H.T
  x_hat = a.Run(w, H, epsilon, seed)
  return x_hat

In [ ]:
def runAHP(data, bin, epsilon):
  domain = (bin,bin)

  seed = 1
  shape_list = [(5,5),(10,10)]
  size = 5000

  w = workload.RandomRange(shape_list=shape_list, 
                        domain_shape=domain, 
                        size=size, 
                        seed=seed)

  a = ahp.ahpND_engine(ratio = 0.85, eta = 0.35)
  H, xedges, yedges = np.histogram2d(data['x'], data['y'], bins = bin)
  H = H.T
  x_hat = a.Run(w, H, epsilon, seed)
  return x_hat

In [ ]:
def runAGrid(data, bin, epsilon):
  domain = (bin,bin)
  seed = 1
  shape_list = [(5,5),(10,10)]
  size = 5000

  w = workload.RandomRange(shape_list=shape_list, 
                        domain_shape=domain, 
                        size=size, 
                        seed=seed)

  a = AG.AG_engine(c = 10, c2 = 5, alpha = 0.5)
  H, xedges, yedges = np.histogram2d(data['x'], data['y'], bins = bin)
  H = H.T
  x_hat = a.Run(w, H, epsilon, seed)
  return x_hat

In [ ]:
from diffprivlib.tools import histogram2d
def runGeoTruncated(data, bin, epsilon):
  newList = histogram2d(data['x'],data['y'],epsilon,bin, density = False)
  x_hat = newList[0].T
  return x_hat



In [ ]:
from diffprivlib.mechanisms.laplace import Laplace

from sys import maxsize
def histogramdd(sample, epsilon, bins, range=None, weights=None, density=None, accountant=None, **unused_args):

    hist, bin_edges = np.histogramdd(sample, bins=bins, range=range, normed=None, weights=weights, density=None)

    dp_mech = Laplace().set_epsilon(epsilon).set_sensitivity(1)
    dp_hist = np.zeros_like(hist)
    iterator = np.nditer(hist, flags=['multi_index'])

    while not iterator.finished:
        dp_hist[iterator.multi_index] = dp_mech.randomise(int(iterator[0]))
        iterator.iternext()

    dp_hist = dp_hist.astype(float, casting='safe')

    if density:
        # calculate the probability density function
        dims = len(dp_hist.shape)
        dp_hist_sum = dp_hist.sum()
        for i in np.arange(dims):
            shape = np.ones(dims, int)
            shape[i] = dp_hist.shape[i]
            # noinspection PyUnresolvedReferences
            dp_hist = dp_hist / np.diff(bin_edges[i]).reshape(shape)

        if dp_hist_sum > 0:
            dp_hist /= dp_hist_sum

  

    return dp_hist, bin_edges


In [ ]:
def laplaceHistogram(array_x, array_y, epsilon, bins, range=None, weights=None, density=None, accountant=None,
                **unused_args):

    try:
        num_bins = len(bins)
    except TypeError:
        num_bins = 1

    if num_bins not in (1, 2):
        xedges = yedges = np.asarray(bins)
        bins = [xedges, yedges]

    hist, edges = histogramdd([array_x, array_y], epsilon=epsilon, bins=bins, range=range, weights=weights,
                              density=density, accountant=accountant)
    
    return hist, edges[0], edges[1]


In [ ]:
def runLaplace(data, bin, epsilon):
  newList = laplaceHistogram(data['x'],data['y'],epsilon,bin, density = False)
  x_hat = newList[0].T
  return x_hat


In [ ]:
seed = 1
domain = (32,32)
shape_list = [(5,5),(10,10)]
size = 5000

w = workload.RandomRange(shape_list=shape_list, 
                        domain_shape=domain, 
                        size=size, 
                        seed=seed)
type(w.query_list[0])

dpcomp_core.query_nd_union.ndRangeUnion

In [ ]:
import pandas as pd

In [ ]:
tasks = ['distribution', 'clusters', 'correlation']
epsilons = [.5, .1, .05, .01]
algorithms = ['DAWA', 'AHP', "AGrid", 'Geometric', 'Laplace']
bins = [32,64]


In [ ]:
chartDictionary = {'0': ['distribution', 'correlation'], '1': ['distribution'], 
                   '2': ['distribution', 'clusters', 'correlation'], '3': ['distribution'],
                   '4': ['distribution', 'clusters'], '5': [ 'correlation'],
                   '6': ['distribution', 'correlation'], '7': ['distribution', 'correlation'],
                   '8':['distribution'], '9': ['distribution'],
                   '10': ['distribution', 'clusters'], '11': ['distribution', 'correlation'],
                   '12': ['correlation'], '13': ['distribution', 'correlation'],
                   '14': ['correlation'], '15': ['distribution'],
                   '16': ['distribution', 'clusters'], '17': ['distribution'],
                   '18': ['clusters'], '19': ['distribution']}

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
def getL2(H, x_hat, bins):
  seed = 1
  domain = (bins,bins)
  shape_list = [(5,5),(10,10)]
  size = 5000

  w = workload.RandomRange(shape_list=shape_list, 
                         domain_shape=domain, 
                         size=size, 
                         seed=seed)


  diff = w.evaluate(H) - w.evaluate(x_hat)
  # print('Per Query Average Absolute Error:', old_div(np.linalg.norm(diff,1), float(diff.size)))
  return(old_div(np.linalg.norm(diff,1), float(diff.size)))

In [ ]:
!pip install pytorch-msssim
from PIL import Image
import numpy as np

import torch
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.colors as colors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def getMSSIM(H, x_hat):
  plt.imshow(H, interpolation='nearest', origin='lower', cmap = 'gray_r', aspect = 'auto',
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
  ax = plt.gca()

  ax.axes.xaxis.set_ticklabels([])
  ax.axes.yaxis.set_ticklabels([])
  plt.savefig('H.png')
  

  plt.imshow(x_hat, interpolation='nearest', origin='lower', cmap = 'gray_r', aspect = 'auto',
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
  ax = plt.gca()

  ax.axes.xaxis.set_ticklabels([])
  ax.axes.yaxis.set_ticklabels([])
  plt.savefig('x_hat.png')

  img = Image.open('H.png')
  X = np.array(img).astype(np.float32)

  img = Image.open('x_hat.png')
  Y = np.array(img).astype(np.float32)
  img_torch = torch.from_numpy(X).unsqueeze(0).permute(0, 3, 1, 2)  # 1, C, H, W
  img_noise_torch = torch.from_numpy(Y).unsqueeze(0).permute(0, 3, 1, 2)
  # print(img_noise_torch)
  msssim_torch = ms_ssim(img_torch, img_noise_torch, win_size=11, data_range=255)

  val = msssim_torch.item()
  os.remove('H.png')
  os.remove('x_hat.png')

  return val


In [ ]:
from scipy.stats import wasserstein_distance
def getEarthMoversDistance(H, x_hat):
  H = H.flatten()
  x_hat = x_hat.flatten()
  return wasserstein_distance(H, x_hat)

In [ ]:
!pip install sdmetrics
!pip install sdv
from pandas._libs.algos import diff_2d
import sdmetrics
metrics = sdmetrics.multi_table.MultiTableMetric.get_subclasses()
from sdv.metrics.tabular import KSTest


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 3.4 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 38.1 MB 67.3 MB/s 
     |████████████████████████████████| 2.5 MB 44.2 MB/s 
     |████████████████████████████████| 45 kB 3.5 MB/s 
     |████████████████████████████████| 15.7 MB 482 kB/s 
     |████████████████████████████████| 11.2 MB 12.7 MB/s 
     |████████████████████████████████| 944 kB 47.7 MB/s 
     |████████████████████████████████| 281 kB 57.5 MB/s 
     |████████████████████████████████| 636 kB 41.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.16.2
    Uninstalling numpy-1.16.2:
      Successfully uninstalled numpy-1.16.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.2.1
    Uninstalling scipy-1.2.1:
      Successfully uninstalled scipy-1.2.1
  Attempting uninstall: py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 4.6 MB/s 
     |████████████████████████████████| 47 kB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 39.7 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [ ]:

def getKSTest(H, x_hat):
  import sdmetrics
  metrics = sdmetrics.multi_table.MultiTableMetric.get_subclasses()
  from sdv.metrics.tabular import KSTest
  # print('hey')
  HDict = {'H': H.flatten()}
  df1 = pd.DataFrame(HDict)
  private = {'H': x_hat.flatten()}
  df2 = pd.DataFrame(private)
  raw_score = KSTest.compute(df1.fillna(0), df2.fillna(0))
  # print(raw_score)
  return raw_score
  # val = KSTest.compute(df1, df2)
  # return val

In [ ]:
def getOneDividedByEpsilon(epsilon):
  return 1/epsilon

In [ ]:
!pip install pyscagnostics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 794 kB 5.2 MB/s 


In [ ]:
from pyscagnostics import scagnostics
import math
def getScagnostics(df, H, xedges, yedges):
  x = df['x'].tolist()
  y = df['y'].tolist()
  newX = []
  newY = []
  H = H.T
  for i in range(len(H)):
    for j in range(len(H[i])):
      for k in range(int(H[i][j])):
        newX.append(np.random.uniform(xedges[i], xedges[i+1]))
        newY.append(np.random.uniform(yedges[j], yedges[j+1]))
  # plt.plot(newX, newY, 'o', color='black');
  measuresOriginal, _ = scagnostics(x, y)
  # print(measuresOriginal)
  measuresPrivate, _ = scagnostics(newX, newY)
  utilityLoss = math.sqrt((abs(measuresOriginal.get('Convex')-measuresPrivate.get('Convex')))**2 
  + (abs(measuresOriginal.get('Skinny')-measuresPrivate.get('Skinny')))**2
  + (abs(measuresOriginal.get('Stringy')-measuresPrivate.get('Stringy')))**2
  + (abs(measuresOriginal.get('Outlying')-measuresPrivate.get('Outlying')))**2
  + (abs(measuresOriginal.get('Skewed')-measuresPrivate.get('Skewed')))**2
  + (abs(measuresOriginal.get('Clumpy')-measuresPrivate.get('Clumpy')))**2
  + (abs(measuresOriginal.get('Sparse')-measuresPrivate.get('Sparse')))**2
  + (abs(measuresOriginal.get('Striated')-measuresPrivate.get('Striated')))**2
  + (abs(measuresOriginal.get('Monotonic')-measuresPrivate.get('Monotonic')))**2
  )

  return utilityLoss

In [ ]:
!python -m pip uninstall matplotlib
!pip install matplotlib==3.1.3

In [ ]:
l2Df = pd.DataFrame(columns = ['Random Query'])
MSSIM =  pd.DataFrame(columns = ['MSSIM'])
EarthMovers = pd.DataFrame(columns = ['EarthMovers'])
OneDividedByEpsilon = pd.DataFrame(columns = ['OneDividedByEpsilon'])
KSTest = pd.DataFrame(columns = ['KSTest'])
Scagnostics = pd.DataFrame(columns = ['Scagnostics'])

from subprocess import run
directory = '/finalData'

for filename in os.scandir(directory):
    if filename.is_file():
        
        # df = pd.read_csv(filename.path)
        for key, value in chartDictionary.items():
          if ('/'+key+'.csv') in (filename.path):
            df = pd.read_csv(filename.path)
        
            for v in value:

              for e in epsilons:
                for b in bins:
                  H, xedges, yedges = np.histogram2d(df['x'], df['y'], bins = b)
                  H = H.T
                  for a in algorithms:
                    if a == 'DAWA':
                      dawaPrivate = runDAWA(df, b,e)
              
                      name = key +'_' + a + '_' + v + '_' + str(e) + '_' + str(b)
                      l2Df.loc[name] = [getL2(H, dawaPrivate, b)]
                      MSSIM.loc[name] = [getMSSIM(H, dawaPrivate)]
                      EarthMovers.loc[name] = [getEarthMoversDistance(H, dawaPrivate)]
                      OneDividedByEpsilon.loc[name] = [getOneDividedByEpsilon(e)]
                      KSTest.loc[name] = [getKSTest(H, dawaPrivate)]
                      Scagnostics.loc[name] = [getScagnostics(df, dawaPrivate, xedges, yedges)]

                    


                     


                    if a == 'AHP':
                      ahpPrivate = runAHP(df,b,e)

                      name = key +'_' + a + '_' + v + '_' + str(e) + '_' + str(b)
                      l2Df.loc[name] = [getL2(H, ahpPrivate, b)]
                      MSSIM.loc[name] = [getMSSIM(H, ahpPrivate)]
                      EarthMovers.loc[name] = [getEarthMoversDistance(H, ahpPrivate)]
                      OneDividedByEpsilon.loc[name] = [getOneDividedByEpsilon(e)]
                      KSTest.loc[name] = [getKSTest(H, ahpPrivate)]
                      Scagnostics.loc[name] = [getScagnostics(df, ahpPrivate, xedges, yedges)]
                     

                    if a == "AGrid":
                      aGridPrivate = runAGrid(df,b,e)
          
                      name = key +'_' + a + '_' + v + '_' + str(e) + '_' + str(b)
                      l2Df.loc[name] = [getL2(H, aGridPrivate, b)]
                      MSSIM.loc[name] = [getMSSIM(H, aGridPrivate)]
                      EarthMovers.loc[name] = [getEarthMoversDistance(H, aGridPrivate)]
                      OneDividedByEpsilon.loc[name] = [getOneDividedByEpsilon(e)]
                      KSTest.loc[name] = [getKSTest(H, aGridPrivate)]
                      Scagnostics.loc[name] = [getScagnostics(df, aGridPrivate, xedges, yedges)]

                    if a == "Geometric":
                      geoPrivate = runGeoTruncated(df,b,e)
                   
                      name = key +'_' + a + '_' + v + '_' + str(e) + '_' + str(b)
                      l2Df.loc[name] = [getL2(H, geoPrivate, b)]
                      MSSIM.loc[name] = [getMSSIM(H, geoPrivate)]
                      EarthMovers.loc[name] = [getEarthMoversDistance(H, geoPrivate)]
                      OneDividedByEpsilon.loc[name] = [getOneDividedByEpsilon(e)]
                      KSTest.loc[name] = [getKSTest(H, geoPrivate)]
                      Scagnostics.loc[name] = [getScagnostics(df, geoPrivate, xedges, yedges)]
                     
                    if a == 'Laplace':
                      laplacePrivate = runLaplace(df,b,e)
                    
                      name = key +'_' + a + '_' + v + '_' + str(e) + '_' + str(b)
                      l2Df.loc[name] = [getL2(H, laplacePrivate, b)]
                      MSSIM.loc[name] = [getMSSIM(H, laplacePrivate)]
                      EarthMovers.loc[name] = [getEarthMoversDistance(H, laplacePrivate)]
                      OneDividedByEpsilon.loc[name] = [getOneDividedByEpsilon(e)]
                      KSTest.loc[name] = [getKSTest(H, laplacePrivate)]
                      Scagnostics.loc[name] = [getScagnostics(df, laplacePrivate, xedges, yedges)]

              


In [ ]:
l2Series = l2Df.squeeze()
MSSIMSeries = MSSIM.squeeze()
EarthMoversSeries = EarthMovers.squeeze()
OneDividedByEpsilonSeries = OneDividedByEpsilon.squeeze()
KSTestSeries = KSTest.squeeze()
ScagnosticsSeries = Scagnostics.squeeze()
finalDF = pd.concat([l2Series, MSSIMSeries, EarthMoversSeries, OneDividedByEpsilonSeries, KSTestSeries, ScagnosticsSeries], axis=1)

finalDF.to_csv('finalUtilityMetrics.csv')  